# Importação e Leitura

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import os
import duckdb
from scipy import stats
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from math import ceil
from math import sqrt


In [ ]:
df = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")
df

## Verificando nulos

In [ ]:
df.isnull().sum().sum() # Muitos valores nulos impossivel exclui-los

In [ ]:
df.isna().sum() # colunas off street name, contributing factor 3 veicle ate o 5 , vehicle type code 3 ate o 5 sao colunas com muitos valores nulos
missing_percent = df.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))

# Lidando Com Outliers

In [ ]:
# Converte os outliars de longitude e latitude em Nan
# lat_min, lat_max = 40.49, 40.92
# lon_min, lon_max = -74.26, -73.69
# filtro_invalidas = (
#     (df['LATITUDE'] < lat_min) | (df['LATITUDE'] > lat_max) |
#     (df['LONGITUDE'] < lon_min) | (df['LONGITUDE'] > lon_max)
# )
# df.loc[filtro_invalidas, ['LATITUDE', 'LONGITUDE']] = np.nan

## Lidando Com Missing Values

In [ ]:
# df.info()

In [ ]:
print("Padronização de Texto e Imputação de Coordenadas")

# 1. Padronização de Texto
text_cols = ['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'BOROUGH']

print("Padronizando nomes de ruas para maiúsculo...")
for col in text_cols:
    # Converte para upper e remove espaços
    df[col] = df[col].astype(str).str.upper().str.strip()
    df[col] = df[col].replace({'NAN': np.nan, 'NONE': np.nan})

missing_start = df['LATITUDE'].isna().sum()
print(f"\nNulos iniciais em LATITUDE: {missing_start:,}")

# PASSO 1: Interseção (Rua A & Rua B)
mask_intersection = (df['ON STREET NAME'].notna()) & (df['CROSS STREET NAME'].notna())
df.loc[mask_intersection, 'intersection_key'] = (
    df.loc[mask_intersection, 'ON STREET NAME'] + " & " + 
    df.loc[mask_intersection, 'CROSS STREET NAME']
)

print("1. Tentando imputar por Interseção...")
for col in ['LATITUDE', 'LONGITUDE']:
    df[col] = df[col].fillna(df.groupby('intersection_key')[col].transform('mean'))

# PASSO 2: Ruas Individuais
street_cols = ['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME']
for street_col in street_cols:
    print(f"2. Tentando imputar por '{street_col}'...")
    for col in ['LATITUDE', 'LONGITUDE']:
        df[col] = df[col].fillna(df.groupby(street_col)[col].transform('mean'))

# PASSO 3: ZIP CODE
if 'ZIP CODE' in df.columns:
    print("3. Tentando imputar por ZIP CODE...")
    for col in ['LATITUDE', 'LONGITUDE']:
        df[col] = df[col].fillna(df.groupby('ZIP CODE')[col].transform('mean'))

# Limpeza final
if 'intersection_key' in df.columns:
    df = df.drop(columns=['intersection_key'])

missing_end = df['LATITUDE'].isna().sum()
recovered = missing_start - missing_end

print(f"Total recuperado: {recovered:,} ({recovered/missing_start:.1%} dos nulos)")
print(f"Restantes sem coordenada: {missing_end:,}")

In [ ]:
# Verifica o padrão dos que sobraram sem Latitude
missing_coords = df[df['LATITUDE'].isna()]

print("Ruas nos dados faltantes:")
print(missing_coords['ON STREET NAME'].value_counts().head())

print("\nZIP CODES nos dados faltantes:")
print(missing_coords['ZIP CODE'].value_counts().head())

In [ ]:
print("Limpeza Final e Filtragem Geográfica")

# 1. Criação do df_cleaned removendo linhas sem coordenada
df_cleaned = df.dropna(subset=['LATITUDE', 'LONGITUDE']).copy()

# 2. Remove pontos que caíram fora
lat_min, lat_max = 40.49, 40.92
lon_min, lon_max = -74.26, -73.69

# Máscara para manter apenas o que está DENTRO de NYC
mask_nyc = (
    (df_cleaned['LATITUDE'] >= lat_min) & (df_cleaned['LATITUDE'] <= lat_max) &
    (df_cleaned['LONGITUDE'] >= lon_min) & (df_cleaned['LONGITUDE'] <= lon_max)
)

outliers = (~mask_nyc).sum()
df_cleaned = df_cleaned[mask_nyc].copy()

print(f"Outliers geográficos removidos: {outliers:,}")
print(f"Total final (df_cleaned): {len(df_cleaned):,}")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

print("Imputação de bairros via Coordenadas (KNN)")

# 1. Diagnóstico Inicial
missing_borough_start = df['BOROUGH'].isna().sum()
print(f"Nulos iniciais em BOROUGH: {missing_borough_start:,}")

# 2. Preparação dos Dados para o KNN
mask_train = df['BOROUGH'].notna() & df['LATITUDE'].notna() & df['LONGITUDE'].notna()
mask_target = df['BOROUGH'].isna() & df['LATITUDE'].notna() & df['LONGITUDE'].notna()

if mask_target.sum() > 0:
    print(f"Treinando KNN com {mask_train.sum():,} registros...")
    
    # Classificador de 1 vizinho mais próximo 
    knn = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
    
    # Treino
    X_train = df.loc[mask_train, ['LATITUDE', 'LONGITUDE']]
    y_train = df.loc[mask_train, 'BOROUGH']
    knn.fit(X_train, y_train)
    
    print(f"Prevendo bairros para {mask_target.sum():,} registros...")
    
    # Prevê os bairros faltantes
    X_target = df.loc[mask_target, ['LATITUDE', 'LONGITUDE']]
    predicted_boroughs = knn.predict(X_target)
    
    # Preenche os nulos no DataFrame original
    df.loc[mask_target, 'BOROUGH'] = predicted_boroughs

# 3. Resultado Final
missing_borough_end = df['BOROUGH'].isna().sum()
recovered = missing_borough_start - missing_borough_end

print(f"Bairros recuperados: {recovered:,}")
print(f"Ainda sem bairro (porque não tinham coordenada): {missing_borough_end:,}")

In [ ]:
print("Preenchendo Colunas de Veículo com 'N/A'")

# Seleção explícita das colunas
vehicle_cols = [
    'VEHICLE TYPE CODE 1', 
    'VEHICLE TYPE CODE 2', 
    'VEHICLE TYPE CODE 3', 
    'VEHICLE TYPE CODE 4', 
    'VEHICLE TYPE CODE 5'
]

# 1. Preenche valores nulos reais (NaN) com 'N/A'
df[vehicle_cols] = df[vehicle_cols].fillna('N/A')

# 2. Garante que tudo é texto e remove 'nan' textual se existir
df[vehicle_cols] = df[vehicle_cols].astype(str).replace(
    {'nan': 'N/A', 'NaN': 'N/A', '<NA>': 'N/A', 'None': 'N/A'}
)

print("\nVerificação (VEHICLE TYPE CODE 1):")
print(df['VEHICLE TYPE CODE 1'].value_counts(dropna=False).head(10))

In [ ]:
print("Preenchimento de Nulos")

# 1. Vítimas: Preencher com 0
victim_cols = ['NUMBER OF PERSONS KILLED', 'NUMBER OF PERSONS INJURED']
for col in victim_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0).astype(int)

# 2. Causas: Preencher com 'Unspecified'
factor_cols = [
    'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
    'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
    'CONTRIBUTING FACTOR VEHICLE 5'
]
df[factor_cols] = df[factor_cols].fillna('Unspecified')

# 3. Ruas e CEP: Preencher com 'N/A'
text_cols = ['ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'ZIP CODE']
df[text_cols] = df[text_cols].fillna('N/A')

# 4. Location: Redundante
if 'LOCATION' in df.columns:
    df = df.drop(columns=['LOCATION'])

print("Nulos restantes por coluna:")
print(df.isna().sum()[df.isna().sum() > 0])

In [ ]:
print("Remoção final")

total_antes = len(df)

# 1. Deleta quem não tem latitude ou longitude
df = df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# 2. Verificação Final
total_depois = len(df)
removidos = total_antes - total_depois

print(f"Linhas removidas: {removidos:,} ({(removidos/total_antes):.1%} do total)")
print(f"Total de linhas restantes: {total_depois:,}")
df_imputed = df

In [ ]:
df.isna().sum()
missing_percent = df.isna().mean() * 100
print(missing_percent.sort_values(ascending=False))

In [ ]:
df_imputed.info()

# Resolvendo inconsistências

In [ ]:
# Identifica colunas do tipo texto (object)
object_cols = df_imputed.select_dtypes(include=['object']).columns

for col in object_cols:
    num_unique = df_imputed[col].nunique()
    print(f"\nColuna: '{col}' | Valores Únicos: {num_unique}")

    # Exibe contagem se for baixa cardinalidade, caso contrário mostra uma amostra
    if num_unique <= 50:
        print(df_imputed[col].value_counts(dropna=False))
    else:
        print(f"Amostra: {df_imputed[col].sample(5, random_state=1).to_list()}")

# Resolvendo Tipos

In [ ]:
df_typed = df_imputed.copy()

# Convertendo colunas que comecam com "NUMBER" para int
number_cols = [col for col in df_typed.columns if col.startswith("NUMBER")]
for col in number_cols:
    df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce').astype('Int64')


df_typed.info()

In [ ]:
# Visualizado colunas de datas
date_cols = [col for col in df_typed.columns if 'DATE' in col.upper() or 'TIME' in col.upper()]
print("Date/Time columns found:")
print(date_cols)
print("\nSample values:")
for col in date_cols:
    print(f"\n{col}:")
    print(df_typed[col].head())

In [ ]:
# Conversão de tipos para data e hora
df_typed['CRASH DATE'] = pd.to_datetime(df_typed['CRASH DATE'], format='%m/%d/%Y', errors='coerce')
df_typed['CRASH TIME'] = pd.to_datetime(df_typed['CRASH TIME'], format='%H:%M', errors='coerce').dt.time

# Criação da coluna de dia da semana
df_typed['DAY_OF_WEEK'] = df_typed['CRASH DATE'].dt.day_name()

# Visualização do resultado
print(df_typed[['CRASH DATE', 'CRASH TIME', 'DAY_OF_WEEK']].head())

In [ ]:
# Seleciona colunas 'object', excluindo 'CRASH TIME' (que é datetime.time)
object_cols = df_typed.select_dtypes(include=['object']).columns.tolist()
if 'CRASH TIME' in object_cols:
    object_cols.remove('CRASH TIME')

# Converte as colunas selecionadas para o tipo 'string' de uma só vez
df_typed[object_cols] = df_typed[object_cols].astype('string')

print(f"{len(object_cols)} colunas convertidas para o tipo string.")

In [ ]:
#versao sem ID e dropando location ja que e a mesma coisa de lat long, alem de renomear o dataset final
df_noid = df_typed.drop(columns=['COLLISION_ID'])
print(df_noid)
df_noid.info()
df_cleaned = df_typed

In [ ]:
#dataset final limpo
df_cleaned

# SQL

In [ ]:
# Carrega dados originais para comparação
df_raw = pd.read_csv("../Motor_Vehicle_Collisions_-_Crashes_20251015.csv")

# Bounding box NYC
lat_min, lat_max = 40.49, 40.92
lon_min, lon_max = -74.26, -73.69

# Identifica registros fora dos limites geográficos no dataset original
raw_outlier_mask = (
    (df_raw['LATITUDE'] < lat_min) | (df_raw['LATITUDE'] > lat_max) |
    (df_raw['LONGITUDE'] < lon_min) | (df_raw['LONGITUDE'] > lon_max)
)

print(f"Comparativo de Registros: Raw ({len(df_raw)}) vs Cleaned ({len(df_cleaned)})")
print(f"Outliers de coordenadas detectados no Raw: {raw_outlier_mask.sum()}")

In [ ]:
df_raw["LATITUDE"]

In [ ]:
# Criar conexão DuckDB em memória se tiver pouca RAM o bagui fica feio

conn = duckdb.connect(':memory:')

# Registrar no DuckDB
conn.register('crashes_raw', df_raw) 
conn.register('crashes_clean', df_cleaned)

print('Tabelas criadas: crashes_raw, crashes_clean')

In [ ]:
queries = {
    'count_rows_raw': "SELECT COUNT(*) AS total_raw FROM crashes_raw;",
    'count_rows_clean': "SELECT COUNT(*) AS total_clean FROM crashes_clean;",

    # Queries para contagem de nulos
    'nulls_raw': """
        SELECT column_name, SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END) AS nulls,
               COUNT(*) AS total, ROUND(100.0*SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END)/COUNT(*),2) AS pct_null
        FROM (
          SELECT CAST("CRASH DATE" AS VARCHAR) AS value, 'CRASH DATE' AS column_name FROM crashes_raw UNION ALL
          SELECT CAST("CRASH TIME" AS VARCHAR), 'CRASH TIME' FROM crashes_raw UNION ALL
          SELECT CAST(LATITUDE AS VARCHAR), 'LATITUDE' FROM crashes_raw UNION ALL
          SELECT CAST(LONGITUDE AS VARCHAR), 'LONGITUDE' FROM crashes_raw UNION ALL
          SELECT CAST("NUMBER OF PERSONS INJURED" AS VARCHAR), 'NUMBER OF PERSONS INJURED' FROM crashes_raw UNION ALL
          SELECT CAST("NUMBER OF PERSONS KILLED" AS VARCHAR), 'NUMBER OF PERSONS KILLED' FROM crashes_raw UNION ALL
          SELECT "CONTRIBUTING FACTOR VEHICLE 1", 'CONTRIBUTING FACTOR VEHICLE 1' FROM crashes_raw UNION ALL
          SELECT "CONTRIBUTING FACTOR VEHICLE 2", 'CONTRIBUTING FACTOR VEHICLE 2' FROM crashes_raw UNION ALL
          SELECT "VEHICLE TYPE CODE 1", 'VEHICLE TYPE CODE 1' FROM crashes_raw UNION ALL
          SELECT "VEHICLE TYPE CODE 2", 'VEHICLE TYPE CODE 2' FROM crashes_raw
        ) t GROUP BY column_name ORDER BY pct_null DESC;""",
    
    'nulls_clean': """
        SELECT column_name, SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END) AS nulls,
               COUNT(*) AS total, ROUND(100.0*SUM(CASE WHEN value IS NULL THEN 1 ELSE 0 END)/COUNT(*),2) AS pct_null
        FROM (
          SELECT CAST("CRASH DATE" AS VARCHAR) AS value, 'CRASH DATE' AS column_name FROM crashes_clean UNION ALL
          SELECT CAST("CRASH TIME" AS VARCHAR), 'CRASH TIME' FROM crashes_clean UNION ALL
          SELECT CAST(LATITUDE AS VARCHAR), 'LATITUDE' FROM crashes_clean UNION ALL
          SELECT CAST(LONGITUDE AS VARCHAR), 'LONGITUDE' FROM crashes_clean UNION ALL
          SELECT CAST("NUMBER OF PERSONS INJURED" AS VARCHAR), 'NUMBER OF PERSONS INJURED' FROM crashes_clean UNION ALL
          SELECT CAST("NUMBER OF PERSONS KILLED" AS VARCHAR), 'NUMBER OF PERSONS KILLED' FROM crashes_clean UNION ALL
          SELECT "CONTRIBUTING FACTOR VEHICLE 1", 'CONTRIBUTING FACTOR VEHICLE 1' FROM crashes_clean UNION ALL
          SELECT "CONTRIBUTING FACTOR VEHICLE 2", 'CONTRIBUTING FACTOR VEHICLE 2' FROM crashes_clean UNION ALL
          SELECT "VEHICLE TYPE CODE 1", 'VEHICLE TYPE CODE 1' FROM crashes_clean UNION ALL
          SELECT "VEHICLE TYPE CODE 2", 'VEHICLE TYPE CODE 2' FROM crashes_clean
        ) t GROUP BY column_name ORDER BY pct_null DESC;""",

    # Top Fatores e Veículos
    'top_factors_raw': """SELECT "CONTRIBUTING FACTOR VEHICLE 1" AS f, COUNT(*) AS c FROM crashes_raw WHERE "CONTRIBUTING FACTOR VEHICLE 1" IS NOT NULL AND TRIM("CONTRIBUTING FACTOR VEHICLE 1") <> '' GROUP BY f ORDER BY c DESC LIMIT 15;""",
    'top_factors_clean': """SELECT "CONTRIBUTING FACTOR VEHICLE 1" AS f, COUNT(*) AS c FROM crashes_clean WHERE "CONTRIBUTING FACTOR VEHICLE 1" IS NOT NULL AND TRIM("CONTRIBUTING FACTOR VEHICLE 1") <> '' GROUP BY f ORDER BY c DESC LIMIT 15;""",
    
    'vehicle_types_raw': """SELECT "VEHICLE TYPE CODE 1" AS v, COUNT(*) AS c FROM crashes_raw WHERE "VEHICLE TYPE CODE 1" IS NOT NULL AND TRIM("VEHICLE TYPE CODE 1") <> '' GROUP BY v ORDER BY c DESC LIMIT 15;""",
    'vehicle_types_clean': """SELECT "VEHICLE TYPE CODE 1" AS v, COUNT(*) AS c FROM crashes_clean WHERE "VEHICLE TYPE CODE 1" IS NOT NULL AND TRIM("VEHICLE TYPE CODE 1") <> '' GROUP BY v ORDER BY c DESC LIMIT 15;""",

    # Agrupamentos Mensais
    'monthly_raw': """SELECT SUBSTRING("CRASH DATE", 7, 4) || '-' || SUBSTRING("CRASH DATE", 1, 2) AS ym, COUNT(*) AS cnt FROM crashes_raw WHERE "CRASH DATE" IS NOT NULL GROUP BY ym ORDER BY ym;""",
    'monthly_clean': """SELECT DATE_TRUNC('month', "CRASH DATE")::VARCHAR AS ym, COUNT(*) AS cnt FROM crashes_clean WHERE "CRASH DATE" IS NOT NULL GROUP BY ym ORDER BY ym;"""
}

# Execução e armazenamento dos resultados
results = {}
for name, q in queries.items():
    results[name] = conn.execute(q).df()
    print(f"-- {name}\n{results[name].head()}\n")

# Variáveis para uso posterior
monthly_raw = results['monthly_raw']
monthly_clean = results['monthly_clean']
nulls_raw = results['nulls_raw']
nulls_clean = results['nulls_clean']
veh_raw = results['vehicle_types_raw']
veh_clean = results['vehicle_types_clean']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#1. Raw vs Clean
miss_compare = nulls_raw[['column_name','pct_null']].merge(
    nulls_clean[['column_name','pct_null']], on='column_name', suffixes=('_raw','_clean')
)

plt.figure(figsize=(10,5))
sns.barplot(data=miss_compare.melt(id_vars='column_name', value_name='pct_null', var_name='dataset'), 
            x='column_name', y='pct_null', hue='dataset')
plt.xticks(rotation=45, ha='right'); plt.title('Percentual de Nulos: Raw vs Clean'); plt.tight_layout(); plt.show()

# 2. Evolução Mensal 
plt.figure(figsize=(12,5))
plt.plot(monthly_raw['ym'], monthly_raw['cnt'], label='Raw', alpha=0.6)
plt.plot(monthly_clean['ym'], monthly_clean['cnt'], label='Clean', alpha=0.6)
plt.xticks(monthly_clean['ym'][::max(len(monthly_clean)//12, 1)], rotation=45)
plt.title('Contagem Mensal de Colisões'); plt.legend(); plt.tight_layout(); plt.show()

# 3. Top Tipos de Veículo 
veh_raw = veh_raw.rename(columns={'v': 'vtype', 'c': 'cnt'})
veh_clean = veh_clean.rename(columns={'v': 'vtype', 'c': 'cnt'})

veh_merge = veh_raw.merge(veh_clean, on='vtype', suffixes=('_raw','_clean'))
veh_sorted = veh_merge.sort_values('cnt_clean', ascending=False).head(15)

plt.figure(figsize=(10,6))
sns.barplot(data=veh_sorted.melt(id_vars='vtype', value_vars=['cnt_raw','cnt_clean']), 
            x='vtype', y='value', hue='variable')
plt.xticks(rotation=45, ha='right'); plt.title('Top 15 Veículos: Raw vs Clean'); plt.ylabel('Contagem'); plt.tight_layout(); plt.show()

# 4. Coordenadas Geográficas 
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

axes[0].scatter(df_raw['LONGITUDE'], df_raw['LATITUDE'], s=1, alpha=0.3)
axes[0].set_title(f'Raw: Com Erros de GPS (n={len(df_raw):,})')
axes[0].set_xlim(-80, -70) # Zoom out
axes[0].set_ylim(35, 45)
axes[0].grid(True, alpha=0.3)

axes[1].scatter(df_cleaned['LONGITUDE'], df_cleaned['LATITUDE'], 
                s=0.1, alpha=0.1, color='#003366') 
axes[1].set_title(f'Clean: Mapa de NYC (n={len(df_cleaned):,})')

axes[1].set_xlim([-74.27, -73.69]) 
axes[1].set_ylim([40.49, 40.92])
axes[1].set_aspect('equal') 
axes[1].grid(True, linestyle=':', alpha=0.4)

plt.tight_layout()
plt.show()
# 5. Heatmap (Fatores x Tipos)
df_heat = conn.execute("""
    SELECT "CONTRIBUTING FACTOR VEHICLE 1" AS factor, "VEHICLE TYPE CODE 1" AS vtype
    FROM crashes_clean
    WHERE factor IS NOT NULL AND vtype IS NOT NULL AND TRIM(factor)<>'' AND TRIM(vtype)<>''
""").df()

top_factors = df_heat['factor'].value_counts().head(10).index
top_vtypes = df_heat['vtype'].value_counts().head(10).index
pivot = pd.crosstab(df_heat[df_heat['factor'].isin(top_factors)]['factor'], 
                    df_heat[df_heat['vtype'].isin(top_vtypes)]['vtype'])

plt.figure(figsize=(12,8))
sns.heatmap(pivot, cmap='Blues', annot=True, fmt='d')
plt.title('Heatmap: Causa do Acidente vs Tipo de Veículo (Top 10)'); plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

# Parte 3 

In [ ]:
print(f"Estrutura do Dataset Limpo: {df_cleaned.shape}")

# Lista colunas e seus tipos de dados
print("\nTipos de dados:")
print(df_cleaned.dtypes)

# Visualização das primeiras linhas
df_cleaned.head()

Reestruturação Necessária

#### Análise de Variáveis Compostas
O dataset atual possui variáveis que violam princípios tidy:
1. **Múltiplos veículos** (VEHICLE TYPE CODE 1-5): formato wide
2. **Múltiplos fatores** (CONTRIBUTING FACTOR 1-5): formato wide  
3. **Múltiplas contagens** (PERSONS/PEDESTRIANS/CYCLISTS/MOTORISTS INJURED/KILLED): podem ser normalizadas

Vamos criar diferentes visões tidy para diferentes análises:

In [ ]:
print("Transformação 1: Veículos (Wide -> Long)")

# Seleção de colunas
base_cols = ['COLLISION_ID', 'CRASH DATE', 'CRASH TIME', 'DAY_OF_WEEK', 'BOROUGH', 'LATITUDE', 'LONGITUDE']
vehicle_type_cols = [c for c in df_cleaned.columns if 'VEHICLE TYPE CODE' in c]

# Melt: Transforma colunas de tipo de veículo em linhas
df_vehicles_long = df_cleaned.melt(
    id_vars=base_cols,
    value_vars=vehicle_type_cols,
    var_name='vehicle_number',
    value_name='vehicle_type'
)

# Extração do índice do veículo e limpeza de nulos
df_vehicles_long['vehicle_number'] = df_vehicles_long['vehicle_number'].str.extract(r'(\d+)').astype(int)

df_vehicles_long = df_vehicles_long[
    df_vehicles_long['vehicle_type'].notna() & 
    (df_vehicles_long['vehicle_type'] != 'N/A')
].sort_values(['COLLISION_ID', 'vehicle_number']).reset_index(drop=True)

print(f"Dataset Tidy (Long): {df_vehicles_long.shape}")
print(f"Colisões únicas: {df_vehicles_long['COLLISION_ID'].nunique():,}")
print(f"Total de veículos envolvidos: {len(df_vehicles_long):,}")

df_vehicles_long.head()

In [ ]:
print("Transformação 2: Fatores Contribuintes (Wide -> Long)")

base_cols = ['COLLISION_ID', 'CRASH DATE', 'CRASH TIME', 'DAY_OF_WEEK', 'BOROUGH', 'LATITUDE', 'LONGITUDE']
factor_cols = [c for c in df_cleaned.columns if 'CONTRIBUTING FACTOR' in c]

df_factors_long = df_cleaned.melt(
    id_vars=base_cols,
    value_vars=factor_cols,
    var_name='vehicle_number',
    value_name='contributing_factor'
)

# Extração do índice do veículo e limpeza
df_factors_long['vehicle_number'] = df_factors_long['vehicle_number'].str.extract(r'(\d+)').astype(int)

# Filtra nulos, 'N/A' e strings vazias
df_factors_long = df_factors_long[
    df_factors_long['contributing_factor'].notna() & 
    (df_factors_long['contributing_factor'] != 'N/A') & 
    (df_factors_long['contributing_factor'].str.strip() != '')
].sort_values(['COLLISION_ID', 'vehicle_number']).reset_index(drop=True)

print(f"Dataset Tidy (Long): {df_factors_long.shape}")
print(f"Colisões únicas: {df_factors_long['COLLISION_ID'].nunique():,}")
print(f"Fatores únicos identificados: {df_factors_long['contributing_factor'].nunique()}")

df_factors_long.head()

In [ ]:
print("Transformação 3: Vítimas (Wide -> Long)")

base_cols = ['COLLISION_ID', 'CRASH DATE', 'CRASH TIME', 'DAY_OF_WEEK', 'BOROUGH', 'LATITUDE', 'LONGITUDE']
casualty_cols = [c for c in df_cleaned.columns if 'NUMBER OF' in c]

df_casualties_long = df_cleaned.melt(
    id_vars=base_cols,
    value_vars=casualty_cols,
    var_name='description',
    value_name='count'
)

# Mantém apenas registros onde houve vítimas (count > 0)
df_casualties_long = df_casualties_long[df_casualties_long['count'] > 0].copy()

# Extrai Tipo e Gravidade usando Regex na descrição da coluna
# Ex: "NUMBER OF PEDESTRIANS INJURED" -> Grupo 1: PEDESTRIANS, Grupo 2: INJURED
pattern = r'NUMBER OF (.+) (INJURED|KILLED)'
extracted = df_casualties_long['description'].str.extract(pattern)
df_casualties_long['victim_type'] = extracted[0]
df_casualties_long['severity'] = extracted[1]

# Remove coluna auxiliar
df_casualties_long = df_casualties_long.drop(columns=['description'])

print(f"Dataset Tidy (Long): {df_casualties_long.shape}")
print(f"Total de vítimas contabilizadas: {df_casualties_long['count'].sum():,.0f}")

print("\nDistribuição por tipo:")
print(df_casualties_long.groupby(['victim_type', 'severity'])['count'].sum().sort_values(ascending=False))

df_casualties_long.head()

In [ ]:
print("Transformação 4: Tabela Principal Tidy")

# Seleção de colunas atômicas (variáveis únicas por colisão)
cols_main = [
    'COLLISION_ID', 'CRASH DATE', 'CRASH TIME', 'DAY_OF_WEEK',
    'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
    'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
    'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
    'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
    'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
    'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED'
]

df_tidy_main = df_cleaned[cols_main].copy()

print(f"Shape Tabela Principal: {df_tidy_main.shape}")
print("\nTipos de dados:")
print(df_tidy_main.dtypes)

# Resumo estatístico
df_tidy_main.describe()

In [ ]:
# Validação da estrutura final
print("Resumo dos Datasets Tidy")
for nome, df in [('Principal', df_tidy_main), ('Veículos', df_vehicles_long), 
                 ('Fatores', df_factors_long), ('Vítimas', df_casualties_long)]:
    print(f"\n[{nome}] Shape: {df.shape} | Duplicatas: {df.duplicated().sum()}")
    print(df.dtypes)

In [ ]:
# Registro dos DataFrames no DuckDB
conn.register('collisions_tidy', df_tidy_main)
conn.register('vehicles_tidy', df_vehicles_long)
conn.register('factors_tidy', df_factors_long)
conn.register('casualties_tidy', df_casualties_long)

# Validação dos registros (Contagens e Somas)
queries = {
    'collisions_tidy (rows)': "SELECT COUNT(*) FROM collisions_tidy",
    'vehicles_tidy (rows)': "SELECT COUNT(*) FROM vehicles_tidy",
    'factors_tidy (rows)': "SELECT COUNT(*) FROM factors_tidy",
    'casualties_tidy (total victims)': "SELECT SUM(count) FROM casualties_tidy"
}

print("Status do Registro no DuckDB")
for label, q in queries.items():
    res = conn.execute(q).fetchone()[0]
    print(f"{label}: {res:,.0f}")

In [ ]:
output_dir = '../processed_data'
os.makedirs(output_dir, exist_ok=True)

datasets = {
    'collisions_tidy.parquet': df_tidy_main,
    'vehicles_tidy.parquet': df_vehicles_long,
    'factors_tidy.parquet': df_factors_long,
    'casualties_tidy.parquet': df_casualties_long,
    'df_cleaned_full.parquet': df_cleaned
}

stats = []
for fname, df in datasets.items():
    fpath = os.path.join(output_dir, fname)
    df.to_parquet(fpath, compression='snappy', index=False)
    
    stats.append({
        'Arquivo': fname,
        'Linhas': len(df),
        'Colunas': df.shape[1],
        'MB': round(os.path.getsize(fpath) / (1024**2), 2)
    })

df_stats = pd.DataFrame(stats)
print(f"Exportação concluída! Tamanho total: {df_stats['MB'].sum():.2f} MB\n")
print(df_stats.to_string(index=False))

In [ ]:
print("Consulta 1: Análise Temporal (Últimos 24 Meses)")

query_temporal = """
WITH monthly_data AS (
    SELECT 
        STRFTIME("CRASH DATE", '%Y-%m') AS year_month,
        COUNT(*) AS collisions,
        SUM("NUMBER OF PERSONS INJURED") AS injured,
        SUM("NUMBER OF PERSONS KILLED") AS killed
    FROM collisions_tidy
    WHERE "CRASH DATE" IS NOT NULL
    GROUP BY year_month
),
with_trends AS (
    SELECT 
        year_month,
        collisions,
        injured,
        killed,
        -- Média Móvel de 3 meses para suavizar tendências
        AVG(collisions) OVER (
            ORDER BY year_month 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS moving_avg_3m,
        -- Comparativo percentual com o mês anterior
        ROUND(100.0 * (collisions - LAG(collisions) OVER (ORDER BY year_month)) / 
              NULLIF(LAG(collisions) OVER (ORDER BY year_month), 0), 2) AS mom_growth_pct
    FROM monthly_data
)
SELECT * FROM with_trends ORDER BY year_month DESC LIMIT 24;
"""

df_temporal = conn.execute(query_temporal).df()

# Visualização Unificada: Evolução de Colisões
plt.figure(figsize=(12, 6))

# Dados reais
plt.plot(df_temporal['year_month'], df_temporal['collisions'], 
         marker='o', linestyle='-', color='#1f77b4', label='Colisões Mensais', alpha=0.8)

# Tendência (Média Móvel)
plt.plot(df_temporal['year_month'], df_temporal['moving_avg_3m'], 
         color='red', linestyle='--', linewidth=2, label='Média Móvel (3 Meses)')

plt.title('Tendência Temporal de Colisões (Últimos 2 Anos)')
plt.ylabel('Volume de Colisões')
plt.xlabel('Mês de Referência')
plt.xticks(rotation=45)
plt.grid(True, linestyle=':', alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
print("Consulta 2: Perfil de Vítimas por bairro")

query_borough = """
SELECT 
    COALESCE(BOROUGH, 'Unknown') AS borough,
    COUNT(*) AS total_collisions,
    SUM("NUMBER OF PEDESTRIANS INJURED") AS ped_injured,
    SUM("NUMBER OF CYCLIST INJURED") AS cyc_injured,
    SUM("NUMBER OF MOTORIST INJURED") AS mot_injured,
    -- Taxa de severidade (feridos por colisão)
    ROUND(AVG("NUMBER OF PERSONS INJURED"), 3) AS avg_severity
FROM collisions_tidy
GROUP BY borough
ORDER BY total_collisions DESC;
"""

df_borough = conn.execute(query_borough).df()

print(df_borough.to_string(index=False))

# Prepara dados para o gráfico (Melt para formato longo compatível com Seaborn)
df_melt = df_borough.melt(
    id_vars='borough', 
    value_vars=['ped_injured', 'cyc_injured', 'mot_injured'],
    var_name='Tipo Vítima', 
    value_name='Qtd Feridos'
)

df_melt['Tipo Vítima'] = df_melt['Tipo Vítima'].replace({
    'ped_injured': 'Pedestres', 
    'cyc_injured': 'Ciclistas', 
    'mot_injured': 'Motoristas'
})

plt.figure(figsize=(12, 6))
sns.barplot(data=df_melt, x='borough', y='Qtd Feridos', hue='Tipo Vítima', palette='viridis')

plt.title('Perfil de Vítimas por Bairro: Quem se fere mais?')
plt.ylabel('Total de Feridos')
plt.xlabel('Bairro')
plt.grid(axis='y', alpha=0.3)
plt.legend(title='Tipo de Vítima')
plt.tight_layout()
plt.show()

In [ ]:
print("Consulta 3: Matriz de Risco (Frequência vs Severidade)")

# Query simplificada para métricas de risco
query_factors = """
SELECT 
    contributing_factor AS factor,
    COUNT(DISTINCT c.COLLISION_ID) AS frequency,
    -- Índice de Severidade: (Feridos + 10x Mortos) / Total Colisões
    ROUND(SUM(c."NUMBER OF PERSONS INJURED" + (c."NUMBER OF PERSONS KILLED" * 10)) * 1.0 / 
          COUNT(DISTINCT c.COLLISION_ID), 2) AS severity_index,
    SUM(c."NUMBER OF PERSONS KILLED") AS total_killed
FROM factors_tidy f
JOIN collisions_tidy c ON f.COLLISION_ID = c.COLLISION_ID
WHERE f.contributing_factor != 'Unspecified'
GROUP BY factor
HAVING frequency > 100 -- Filtro de relevância estatística
ORDER BY frequency DESC;
"""

df_risk = conn.execute(query_factors).df()

print(f"Fatores analisados: {len(df_risk)}")
print(df_risk.head(10).to_string(index=False))

plt.figure(figsize=(14, 8))

# X: Frequência (Log), Y: Severidade, Cor/Tamanho: Total de Mortos
scatter = plt.scatter(
    df_risk['frequency'], 
    df_risk['severity_index'], 
    s=df_risk['total_killed'] * 5 + 50, #
    c=df_risk['severity_index'],        
    cmap='RdYlGn_r',                  
    alpha=0.7, 
    edgecolors='grey'
)

plt.xscale('log')
plt.title('Matriz de Risco: Frequência vs. Severidade dos Fatores')
plt.xlabel('Frequência de Ocorrência (Escala Log)')
plt.ylabel('Índice de Severidade (Impacto por Acidente)')
plt.grid(True, which="both", ls="--", alpha=0.3)


for _, row in df_risk.nlargest(3, 'frequency').iterrows():
    plt.text(row['frequency'], row['severity_index'], row['factor'][:20], 
             fontsize=9, ha='right', va='bottom', fontweight='bold')


for _, row in df_risk.nlargest(3, 'severity_index').iterrows():
    plt.text(row['frequency'], row['severity_index'], row['factor'][:20], 
             fontsize=9, ha='left', va='top', color='red', fontweight='bold')

plt.colorbar(scatter, label='Índice de Severidade')
plt.tight_layout()
plt.show()

In [ ]:
print("Consulta 4: Padrões Temporais (Heatmap Semanal)")

query_time = """
SELECT 
    "DAY_OF_WEEK" AS day_of_week,
    EXTRACT(HOUR FROM "CRASH TIME") AS hour,
    COUNT(*) AS collisions
FROM collisions_tidy
WHERE "CRASH TIME" IS NOT NULL
GROUP BY day_of_week, hour;
"""

df_time = conn.execute(query_time).df()

# Tradução e Ordenação
day_map = {
    'Sunday': 'Domingo', 'Monday': 'Segunda', 'Tuesday': 'Terça',
    'Wednesday': 'Quarta', 'Thursday': 'Quinta', 'Friday': 'Sexta', 'Saturday': 'Sábado'
}
day_order = ['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado']

# Cria a matriz (Pivot Table): Linhas = Horas, Colunas = Dias
pivot_table = df_time.pivot_table(
    index='hour', 
    columns='day_of_week', 
    values='collisions', 
    aggfunc='sum'
).rename(columns=day_map).reindex(columns=day_order)

plt.figure(figsize=(12, 10))

sns.heatmap(
    pivot_table, 
    cmap='YlOrRd',      
    annot=True,         
    fmt='.0f',         
    linewidths=.5,     
    cbar_kws={'label': 'Volume de Colisões'}
)

plt.title('Mapa de Calor: Frequência de Acidentes (Dia x Hora)')
plt.ylabel('Hora do Dia')
plt.xlabel('Dia da Semana')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### teste

In [2]:
print("Consulta 6: Mapa Simples de Hotspots")

query_hotspots = """
SELECT 
    ROUND(LATITUDE, 3) AS lat,
    ROUND(LONGITUDE, 3) AS lon,
    COALESCE(BOROUGH, 'Desconhecido') AS borough,
    COUNT(*) AS collisions
FROM collisions_tidy
WHERE LATITUDE BETWEEN 40.49 AND 40.92 
  AND LONGITUDE BETWEEN -74.26 AND -73.69
GROUP BY lat, lon, borough
HAVING collisions >= 10
ORDER BY collisions DESC
LIMIT 100;
"""

df_hotspots = conn.execute(query_hotspots).df()
print(df_hotspots.head().to_string(index=False))

plt.figure(figsize=(10, 8))

sns.scatterplot(
    data=df_hotspots, 
    x='lon', 
    y='lat', 
    hue='borough',  # Colore por região para facilitar a leitura
    palette='bright',
    s=60,           # Tamanho fixo dos pontos
    alpha=0.7       # Leve transparência
)

plt.title('Distribuição Geográfica dos 100 Maiores Pontos de Acidente')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True, linestyle=':', alpha=0.5)
plt.legend(title='Região', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

Consulta 6: Mapa Simples de Hotspots


NameError: name 'conn' is not defined

In [ ]:

print("Derivação de Variáveis Auxiliares")

# 1. Padronização de Tempo
# Garante que CRASH TIME é string para extração segura, depois converte para data
time_objs = pd.to_datetime(df_tidy_main['CRASH TIME'].astype(str), format='%H:%M:%S', errors='coerce')
df_tidy_main['hour'] = time_objs.dt.hour.fillna(0).astype(int) # Preenche nulos com 0 por segurança

# 2. Variáveis de Calendário
df_tidy_main['day_of_week'] = df_tidy_main['DAY_OF_WEEK']
df_tidy_main['is_weekend'] = df_tidy_main['day_of_week'].isin(['Saturday', 'Sunday'])

# 3. Classificação de Período (Vetorizada com np.select)
cond_night = df_tidy_main['hour'].isin([22, 23, 0, 1, 2, 3, 4, 5])
cond_peak = df_tidy_main['hour'].isin([7, 8, 9, 17, 18, 19])

df_tidy_main['hour_period'] = np.select(
    [cond_night, cond_peak], 
    ['Noite/Madrugada', 'Pico'], 
    default='Intermediário'
)

# 4. Métricas de Severidade
# Colunas totais diretamente para evitar dupla contagem.
df_tidy_main['fatalities_total'] = df_tidy_main['NUMBER OF PERSONS KILLED'].fillna(0)
df_tidy_main['injuries_total'] = df_tidy_main['NUMBER OF PERSONS INJURED'].fillna(0)

# Índice de Severidade (Peso 5 para mortes)
df_tidy_main['severity_index'] = df_tidy_main['injuries_total'] + (df_tidy_main['fatalities_total'] * 5)

cols_view = ['day_of_week', 'is_weekend', 'hour', 'hour_period', 'fatalities_total', 'severity_index']
print(df_tidy_main[cols_view].head())

In [ ]:
print("Análise Univariada")

fig, axes = plt.subplots(2, 3, figsize=(16, 8))
ax = axes.flatten()

# 1. Horário 
sns.countplot(data=df_tidy_main, x='hour', ax=ax[0], color='#4c72b0')
ax[0].set_title('Distribuição de Acidentes por Hora')

# 2. Período do Dia
sns.countplot(data=df_tidy_main, x='hour_period', ax=ax[1], palette='Set2')
ax[1].set_title('Concentração por Período')

# 3. Dia da Semana
order_days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
sns.countplot(data=df_tidy_main, x='day_of_week', order=order_days, ax=ax[2], palette='viridis')
ax[2].tick_params(axis='x', rotation=45)
ax[2].set_title('Distribuição Semanal')

# 4. Distribuições de Severidade e Vítimas
# Nota: Escala Log no eixo Y porque a distribuição é muito assimétrica (muitos zeros)
vars_hist = [
    ('severity_index', 'Índice de Severidade', '#dd8452'),
    ('fatalities_total', 'Fatalidades Totais', '#55a868'),
    ('injuries_total', 'Feridos Totais', '#c44e52')
]

for i, (col, title, color) in enumerate(vars_hist):
    idx = i + 3
    sns.histplot(df_tidy_main[col], ax=ax[idx], bins=20, color=color, kde=False)
    ax[idx].set_yscale('log')  # Essencial para visualizar caudas longas
    ax[idx].set_title(f'{title} (Escala Log)')
    ax[idx].set_ylabel('Frequência (Log)')

plt.tight_layout()
plt.show()

# Estatísticas Consolidadas
cols_stats = ['fatalities_total', 'injuries_total', 'severity_index']

# Calcula estatísticas descritivas
desc = df_tidy_main[cols_stats].describe().T

# Adiciona Skew e Kurtosis
desc['skew'] = df_tidy_main[cols_stats].skew()
desc['kurtosis'] = df_tidy_main[cols_stats].kurt()

# Calcula Outliers
Q1 = df_tidy_main[cols_stats].quantile(0.25)
Q3 = df_tidy_main[cols_stats].quantile(0.75)
IQR = Q3 - Q1
outlier_mask = (df_tidy_main[cols_stats] < (Q1 - 1.5 * IQR)) | (df_tidy_main[cols_stats] > (Q3 + 1.5 * IQR))
desc['outliers_count'] = outlier_mask.sum()

print("\nResumo Estatístico e Distribuição:")
print(desc[['mean', 'std', 'max', 'skew', 'kurtosis', 'outliers_count']].round(3).to_string())

In [ ]:
print("Análise Bivariada e Correlações")

# 1. Matriz de Correlação
cols_corr = ['fatalities_total', 'injuries_total', 'severity_index', 'hour']
corr = df_tidy_main[cols_corr].corr(method='spearman')

plt.figure(figsize=(6, 5))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlação Spearman')
plt.show()

# 2. Severidade por hora
hour_stats = df_tidy_main.groupby('hour')[['severity_index', 'injuries_total', 'fatalities_total']].mean().reset_index()

plt.figure(figsize=(10, 4))
sns.lineplot(data=hour_stats, x='hour', y='severity_index', marker='o', color='#c44e52')
plt.title('Severidade Média por Hora do Dia')
plt.ylabel('Índice Médio de Severidade')
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 24))
plt.show()

# 3. Severidade por bairro
borough_stats = df_tidy_main.groupby('BOROUGH').agg({
    'severity_index': 'mean',
    'fatalities_total': 'mean',
    'injuries_total': 'mean',
    'COLLISION_ID': 'count'
}).rename(columns={'COLLISION_ID': 'total_collisions'}).reset_index()

borough_stats = borough_stats.sort_values('severity_index', ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(data=borough_stats, x='BOROUGH', y='severity_index', palette='magma')
plt.title('Severidade Média por Bairro (Borough)')
plt.xticks(rotation=45)
plt.ylabel('Índice Médio de Severidade')
plt.show()

print("Estatísticas por Borough:")
print(borough_stats.round(4).to_string(index=False))

## TESTE DE HIPOTESES

In [ ]:
from scipy.stats import chi2_contingency

print("Teste de Hipóteses")

results = []

# H1: Letalidade Noturna (22h-05h)
df_tidy_main['is_night'] = df_tidy_main['hour'].isin([22, 23, 0, 1, 2, 3, 4, 5])
df_tidy_main['has_fatality'] = df_tidy_main['fatalities_total'] > 0

# Teste Chi-Quadrado
tab_h1 = pd.crosstab(df_tidy_main['is_night'], df_tidy_main['has_fatality'])
chi2, p_val_h1, _, _ = chi2_contingency(tab_h1)

rate_night = tab_h1.loc[True, True] / tab_h1.loc[True].sum()
rate_day = tab_h1.loc[False, True] / tab_h1.loc[False].sum()

accepted_h1 = (p_val_h1 < 0.05) and (rate_night > rate_day)

results.append({
    'Hipótese': 'H1: Maior letalidade noturna',
    'Taxa A (Noite)': f"{rate_night*100:.3f}%",
    'Taxa B (Dia)': f"{rate_day*100:.3f}%",
    'P-Valor': p_val_h1,
    'Resultado': 'ACEITA' if accepted_h1 else 'REJEITADA'
})

# H2: Perfil Horário de Pico (07-09h, 17-19h)
peak_hours = [7, 8, 9, 17, 18, 19]
df_tidy_main['is_peak'] = df_tidy_main['hour'].isin(peak_hours)

h2_stats = df_tidy_main.groupby('is_peak')[['injuries_total', 'fatalities_total']].mean()

injuries_peak = h2_stats.loc[True, 'injuries_total']
injuries_off = h2_stats.loc[False, 'injuries_total']
fatal_peak = h2_stats.loc[True, 'fatalities_total']
fatal_off = h2_stats.loc[False, 'fatalities_total']

# Critério: Pico deve ter MAIS feridos e MENOS mortes (acidentes leves)
accepted_h2 = (injuries_peak > injuries_off) and (fatal_peak < fatal_off)

results.append({
    'Hipótese': 'H2: Pico (+Feridos / -Mortos)',
    'Taxa A (Pico)': f"Inj:{injuries_peak:.3f} | Fat:{fatal_peak:.4f}",
    'Taxa B (Fora)': f"Inj:{injuries_off:.3f} | Fat:{fatal_off:.4f}",
    'P-Valor': 'N/A',
    'Resultado': 'ACEITA' if accepted_h2 else 'REJEITADA'
})

# Resultado Final
print(pd.DataFrame(results).to_string(index=False))

if accepted_h1:
    print(f"\n[H1] A noite é {rate_night/rate_day:.1f}x mais letal que o dia.")
if accepted_h2:
    print("[H2] Horários de pico têm maior frequência de feridos, mas menor letalidade.")